In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import sys
sys.executable

import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
base_dir = 'fruit_photos'
IMAGE_SIZE = 224
BATCH_SIZE = 64

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(base_dir, rescale=1./255, validation_split=0.20)

## Extract and Preprocess all the data

In [4]:
train_gen = data_generator.flow_from_directory(base_dir,
                                               target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                               batch_size=BATCH_SIZE,
                                               subset='training')
val_gen = data_generator.flow_from_directory(base_dir,
                                             target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                             batch_size=BATCH_SIZE,
                                             subset='validation')

Found 56445 images belonging to 15 classes.
Found 14104 images belonging to 15 classes.


In [5]:
for image_batch, label_batch in train_gen:
    break
image_batch.shape, label_batch.shape # The shape of input tensors

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


((64, 224, 224, 3), (64, 15))

In [6]:
#Create a labels file to be exported into TFLite
print(train_gen.class_indices) # The lables
#Format them
labels = '\n'.join(train_gen.class_indices.keys())

with open('labels.txt', 'w') as f:
    f.write(labels)

{'Apple': 0, 'Banana': 1, 'Carambola': 2, 'Guava': 3, 'Kiwi': 4, 'Mango': 5, 'Orange': 6, 'Peach': 7, 'Pear': 8, 'Persimmon': 9, 'Pitaya': 10, 'Plum': 11, 'Pomegranate': 12, 'Tomatoes': 13, 'muskmelon': 14}


In [7]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 4s 0us/step


## Build Model

In [8]:
# Freeze the base model and fine tune by stacking layers on top
base_model.trainable = False

In [9]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 2, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(15, activation='softmax')
])

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 6, 6, 32)          163872    
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 15)                495       
Total params: 2,422,351
Trainable params: 164,367
Non-trainable params: 2,257,984
_________________________________________________________________


## Train just the last few layers created

In [ ]:
epochs = 10

history = model.fit_generator(train_gen,
                              epochs=epochs,
                              verbose=1,
                              validation_data=val_gen)

Epoch 1/10
  8/882 [..............................] - ETA: 2:23:21 - loss: 2.8904 - accuracy: 0.0547

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Fine tune the model by training more layers

In [ ]:
base_model.trainable = True

In [ ]:
print("# Layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_layer = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_layer]: layer.trainable =  False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

model.summary()

In [ ]:
fine_tune = model.fit_generator(train_gen, epochs=5, validation_data=val_gen)